# Motor Imagery EEG-BCIs - 0 to Deep Learning with BCI-IV 2a dataset

@author: João Araújo ([LinkedIn](https://www.linkedin.com/in/joao-araujo-60470193/))

## 3 - Intro to Deep Learning for BCIs: From Filterbank C(NN)SP to EEGNet
Let's combine the previous conceptual ideas of what is going on on our EEG data to form an hypothesis that can guide our Machine Learning modelling strategy. When imagining different motor movements, a subject's EEG changes so that:

- **Temporal Changes**: Specific rhythms of brain oscillations change (like ERS or ERD)
- **Spatial Changes**: These changes happen differently across hemispheres or EEG channels.
- **Classifier**: These changes might or might not have a linear relationship with MI class.

For this Notebook, I also want to provide some evidence that we do not need so much pre-processing when applying these Deep Learning methods. So I encourage you to re-run the first Notebook after editing the last cell line:


```
# Get the event EEG data
event_data = X_filtered[eeg_channels,triggers[i]:triggers[i] + mi_duration]
```
to:

```
# Get the event EEG data
event_data = X[eeg_channels,triggers[i]:triggers[i] + mi_duration]
```
Once that is done and we have our new .mat, let's start with the imports and load functions:

In [1]:
# Imports
from scipy.io import loadmat
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from keras.constraints import MaxNorm
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D,BatchNormalization, Dropout, Conv2D, Dense, Flatten, Input, Activation,AveragePooling2D, MaxPooling2D
from tensorflow import keras
import os
import gc

# Load our data
x = loadmat("BCI_IV_2a.mat")
y = np.squeeze(x['class_labels'])-1
subj_labels = x['subj_labels']
M_res = x['M_res']
n_channels = np.size(M_res,axis = 2)
del x

# In case you have CUDA compatibility
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

## Filterbank C(NN)SP
We have previously covered CSP as a way to create spatial filters in a supervised manner. Filterbank CSP is an extension of the CSP methodology as it calculates CSP not for a single temporal filter, but for a filterbank (hence the name, I guess). This is a particularly useful method if we believe that different oscillation bands (alpha, beta, gamma, etc.) hold different spatial patterns between different MI classes. I have previously covered the reugular version of Filterbank CSP [here](https://github.com/joaoaraujo1/ML-BCI-dryEEG), so you can check it out for more details.

Filterbank CSP however, suffers from a limitation: We need to define our bands of interest by hand, design the bandpass filters ourselves or choose an existing architecture. This means that we need to have previous knowledge about the frequency bands where the important information might be. That can obviously limit our model's accuracy if we are not right about which specific oscillatory bands can be useful or if we missed some bands. We can, however, work around this by implementing the same ideas of Filterbank CSP in a Convolutional Neural Network architecture, integrate it with a Logistic Regression classifier and train the whole thing in the same integrated optimization process. Here's how we do it using Tensorflow and Keras:  

In [2]:
"""
FbC(NN)SP
Filterbank CSP implemented as a CNN architecture
Takes dictionary "params" as input:
    params['t_filter_n']: number of temporal filters to calculate
    params['t_filter_order']: order (kernel size-1) of the temporal filters
    params['s_filter_n']: number of spatial filters to calculate
    params['avg_samples']: number of timepoint samples to average oscillatory power

"""

def FBC_NN_SP(X: np.ndarray, params : dict, n_classes: int) -> keras.Model:
    
    ## Input and formatting ##
    n_channels = X.shape[1]

    input_shape = X.shape[1:]
    
    inputLayer = Input(input_shape)
    
    reshapeLayer = tf.expand_dims(inputLayer,axis=-1)  
    
    ## Temporal filtering ##
    conv2dLayer_1 = Conv2D(filters = params['t_filter_n'], 
                            kernel_size= (1,params['t_filter_order']+1), 
                            activation='linear',
                            use_bias = False,
                            padding = 'valid')(reshapeLayer) 
    
    ## Spatial filtering ##
    depthconvLayer = DepthwiseConv2D(kernel_size= (n_channels,1), 
                                     activation='linear',
                                     padding = 'valid',
                                     depth_multiplier=params['s_filter_n'],
                                     use_bias = False)(conv2dLayer_1) 
    
    
    ## Signal Power/Variance estimation ##
    activationLayer_1 = Activation(activation = tf.math.square)(depthconvLayer)
    poolLayer_1 = AveragePooling2D(pool_size=(1, conv2dLayer_1.shape[2]))(activationLayer_1) 
    
    ## Logistic/Softmax Regression ##
    flatFeatures = Flatten()(poolLayer_1)
    if n_classes == 2:
        outputLayer = Dense(units = 1,activation='sigmoid')(flatFeatures)
    else:
        outputLayer = Dense(units = n_classes,activation='softmax')(flatFeatures)        
    
    model = keras.Model(inputLayer,outputLayer)
    return model

Let's have an overview of each sequential operation on this model:

0) **Input**: We input the model our `n_channels x n_timepoints matrix`.

1) **Temporal Filtering**: Convolution across the temporal dimension of the signal. [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) is doing the work of a bandpass filter in a sense. We have to choose the number of filters and the order of the filter (but not the bandpass frequencies). Output shape: `t_filter_n x (n_timepoints - t_filter_order)`

2) **Spatial Filtering**: Convolution across the EEG channel dimension. Mathematically this is similar to a CSP projection. We use [DepthwiseConv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/DepthwiseConv2D) and not Conv2D, allowing each spatial filter to be tailored to its respective temporal filter. We can choose the number of spatial filters per temporal filter with `s_filter_n`. Output shape: `(t_filter_n * s_filter_n) x (n_timepoints - t_filter_order)`.

3) **Signal Power/Variance estimation**: By using the square activation function followed by an average of the signal we are taking the power or basically the variance if the signal's mean = 0. These summary statistics of the temporally and spatially filtered signal will be used as the final features for our logistic classifier. Output shape: `(t_filter_n * s_filter_n) x 1`

4) **Logistic/Softmax Regression**: We use [Dense](https://keras.io/api/layers/core_layers/dense/) to create a single fully connected layer to our logistic / softmax decision neuron(s).

Let's print our model's structure, defining 4 temporal filters of order 4 with 2 spatial filters each:

In [3]:
# Filterbank C(NN)SP parameters
params = {
    "t_filter_n": 4,
    "t_filter_order":4,
    "s_filter_n" : 2
    }

# Define the model
fbcnnsp = FBC_NN_SP(X=M_res, params=params, n_classes=len(np.unique(y)))

# Print model
print(fbcnnsp.summary())                    

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 22, 1000)]        0         
                                                                 
 tf.expand_dims (TFOpLambda)  (None, 22, 1000, 1)      0         
                                                                 
 conv2d (Conv2D)             (None, 22, 996, 4)        20        
                                                                 
 depthwise_conv2d (Depthwise  (None, 1, 996, 8)        176       
 Conv2D)                                                         
                                                                 
 activation (Activation)     (None, 1, 996, 8)         0         
                                                                 
 average_pooling2d (AverageP  (None, 1, 1, 8)          0         
 ooling2D)                                                   

So here you go - an implementation of Filterbank CSP with a flavour of CNNs. While this is approach is interesting, it should be pointed out that:

1) This model lacks a lot of ability to deal with nonlinear relationships between data and classifications. If we never used the *square* activation function, **our model could be summarized as a sequence of linear transformations**!

2) When we average each spatiotemporal trial representation into a single summary statistic, we lose the ability to assess if earlier or later stages of the trial have different levels of importance - everything gets **dilluted into one single feature**.

So let's go one step further and explore a famous CNN architecture for general EEG classification.

## EEGNet
EEGNet ([paper](https://arxiv.org/abs/1611.08024)) takes the ideas of Filterbank CSP and adds another layer of complexity to it, check it out:

In [4]:
"""
EEGNet 
F1: Temporal filters
D: number of spatial filters to learn per temporal filter F1
F2: Compression (if F2 < D*F1) / Overcompletion (if F2 > D*F1) filters

"""


def EEGNet(X : np.ndarray ,F1 : int ,D : int ,F2 : int ,n_classes : int) -> keras.Model:
    
    ##### Block 1 #####
    n_channels = X.shape[1]
    
    input_shape = X.shape[1:]
    
    inputLayer = Input(input_shape)
    
    reshapeLayer = tf.expand_dims(inputLayer,axis=-1)
    
    conv2dLayer_1 = Conv2D(filters = F1, 
                            kernel_size= (1,64), 
                            activation='linear',
                            use_bias = False,
                            padding = 'same')(reshapeLayer) 
    
    normLayer_1 = BatchNormalization()(conv2dLayer_1)
    
    depthconvLayer = DepthwiseConv2D(kernel_size= (n_channels,1), 
                                     activation='linear',
                                     padding = 'valid',
                                     depth_multiplier= D,
                                     use_bias = False,
                                     kernel_constraint = MaxNorm(max_value = 1))(normLayer_1) 
    
    normLayer_2 = BatchNormalization()(depthconvLayer)
    
    activationLayer_1 = Activation(activation = 'elu')(normLayer_2)
    
    poolLayer_1 = AveragePooling2D(pool_size=(1, 4))(activationLayer_1)
    
    dropLayer_1 = Dropout(rate = 0.25)(poolLayer_1) # 0.5 for within-subj
    
    
    
    ##### Block 2 #####
    
    separableconvLayer = SeparableConv2D(filters=F2,
                                         use_bias = 0,
                                         kernel_size= (1,16), 
                                         activation='linear',
                                         padding = 'same')(dropLayer_1)
    
    normLayer_3 = BatchNormalization()(separableconvLayer)
    
    activationLayer_2 = Activation(activation = 'elu')(normLayer_3)
    
    poolLayer_2 = AveragePooling2D(pool_size=(1, 8))(activationLayer_2)
    
    dropLayer_2 = Dropout(rate = 0.25)(poolLayer_2) # 0.5 for within-subj
    
        
    ####
    
    flatFeatures = Flatten()(dropLayer_2)
    if n_classes == 2:
        outputLayer = Dense(units = 1,activation='sigmoid',kernel_constraint=MaxNorm(max_value=2))(flatFeatures)
    else:
        outputLayer = Dense(units = n_classes,activation='softmax',kernel_constraint=MaxNorm(max_value=2))(flatFeatures)        
    
    model = keras.Model(inputLayer,outputLayer)
    return model

While the first block is somewhat similar to Filterbank C(NN)SP, the main difference is the addition of a second block of operations that both filters each channel separately and mixes the result of these output channels. This is achieved through the [SeparableConv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/SeparableConv2D) operation. The temporal filter order is fixed at 64 (half of the sampling frequency of the datasets the model was validated on), but we still need to choose the number of temporal filters (**F1**), spatial filters (**D**) and final mixed convolutions (**F2**). Other than that you can notice:

1) Addition of 2 exponential linear activations to add non-linearity to the model;

2) Usage of [Dropout](https://keras.io/api/layers/regularization_layers/dropout/) as a regularization method (i.e. preventing overfitting);

3) Sequential pooling of short sequences of the trial, allowing multiple features across different timepoints of each trial (versus single feature for Filterbank CSP)

4) [Batch Normalization](https://keras.io/api/layers/normalization_layers/batch_normalization/) to help model convergence.

Let's see how the transformations of a complete EEGNet with 8 temporal filters, 2 spatial filters per temporal filter looks like:

In [5]:
# Define hyperparameters
F1 = 8
D = 2
F2 = F1*D

# Define model
eegnet_82 = EEGNet(X=M_res ,F1 = F1 ,D = D ,F2 = F2 ,n_classes = 4) # EEGNet8,2

# Check model summary
print(eegnet_82.summary())   

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 22, 1000)]        0         
                                                                 
 tf.expand_dims_1 (TFOpLambd  (None, 22, 1000, 1)      0         
 a)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 1000, 8)       512       
                                                                 
 batch_normalization (BatchN  (None, 22, 1000, 8)      32        
 ormalization)                                                   
                                                                 
 depthwise_conv2d_1 (Depthwi  (None, 1, 1000, 16)      352       
 seConv2D)                                                       
                                                           

### Model Validation 1 - Validation split of training set for early stoppage
When training these architectures is good to put in place the best strategies to avoid overfitting and getting the best of our models. One of those strategies is splitting our training set so that part of it is used to validate each model in each fold on the fly and stop training before we overfit. In this implementation we can choose to split our validation set in `k` parts and we hold out a proportion `1/k` of the dataset to test it after every epoch:

In [6]:
## Splitting training data into validation train/test splits
## A proportion 1/k data is used for validation test
def validationSplit(subj_labels,g_splits,i,k=3):
    # Get our unique subjects
    subject_splits = np.unique(subj_labels[g_splits[i][0]])
    # Shuffle our unique subjects array
    np.random.shuffle(subject_splits)
    # Split them in k parts
    splits = np.array_split(subject_splits,k)
    # Use k-1 parts to train and 1 part to test
    subj_test_val = splits[-1]
    subj_train_val = np.concatenate(splits[:k-1])
    # return train/test subject label arrays
    return subj_train_val, subj_test_val

### Model Validation 2 - Group K-Fold for subjectwise classification
If we want our predictions to generalize across subjects we should make sure that at every fold, our test data is gathered from a different subject. Like before, we use Leave-One-Participant-Out cross-validation. In Keras, we can get those cross-validation splits easily by:

In [7]:
group_kfold = GroupKFold(n_splits=9) # 9 participants total
g_splits = list(group_kfold.split(M_res, y, subj_labels))

### Model Validation 3 - Boost your model validation speed with GPU (Optional)
If you have access to a GPU that supports CUDA you can further optimize its memory management:

In [8]:
## Memory Management ##
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

## Model Comparisons
We are now ready to test our two models and compare them. Will EEGNet beat the classic Filterbank CSP reinvented as a CNN? (This cell might take a while to run depending on your hardware availability, feel free to get a healthy snack)

In [9]:
model_type = 1 # 0 - FBC(NN)SP, 1- EEGNet

## Useful variables to collect for AUC and accuracy
total_acc = [] 
full_scores = []

for i,(train_idc, test_idc) in enumerate(g_splits):
    
    # Get our test set and labels
    X_test = M_res[test_idc,:,:]
    y_test = y[test_idc] 
    
    # Callback for early stopping
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=10,
                                                restore_best_weights = True)
    
    if model_type == 0:
        model = FBC_NN_SP(X=M_res, params=params, n_classes=4) # FBC(NN)SP
    elif model_type == 1:
        model = EEGNet(X=M_res ,F1 = F1 ,D = D ,F2 = F2 ,n_classes = 4) # EEGNet8,2
    
    # Choose loss function to optimize
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=keras.optimizers.Adam(learning_rate = 0.001),
                  metrics=["accuracy"])
    
    # Split training set to validation sets for early stoppage
    subj_train_val, subj_test_val = validationSplit(subj_labels,g_splits,i,3)

    X_train_val = M_res[ np.isin(subj_labels,subj_train_val),:,:]
    y_train_val = y[np.isin(subj_labels,subj_train_val)]
    X_test_val = M_res[ np.isin(subj_labels,subj_test_val),:,:]
    y_test_val = y[np.isin(subj_labels,subj_test_val)]

    # Fit
    history = model.fit(x=X_train_val,
                        y=y_train_val,
                        epochs=100,
                        batch_size = 32, 
                        shuffle = True,
                        validation_data = [X_test_val,y_test_val],
                        verbose = 0,
                        callbacks = [callback])


    # Classification as the highest proability prediction for test data
    y_probs = model.predict(X_test)
    y_pred = np.argmax(y_probs, axis = 1)

    # Get accuract score
    accuracy = accuracy_score(y_test, y_pred)
    print('Subject %d accuracy: %.2f%%' %(i+1,accuracy*100))
    i += 1

    # Save each fold's accuracy
    full_scores.append(accuracy)

    ## Memory Management ##
    del model, history
    gc.collect()

    
# Save mean accuracy
total_acc.append([np.mean(full_scores),np.var(full_scores)])

# Print
print('Model accuracy percent: %.2f%% (var:%.2f%%)' %(total_acc[0][0]*100,total_acc[0][1]*100))

18/18 [==============================] - 0s 11ms/step
Subject 1 accuracy: 63.18%
18/18 [==============================] - 0s 12ms/step
Subject 2 accuracy: 50.27%
18/18 [==============================] - 0s 11ms/step
Subject 3 accuracy: 29.93%
17/17 [==============================] - 0s 15ms/step
Subject 4 accuracy: 70.17%
17/17 [==============================] - 0s 15ms/step
Subject 5 accuracy: 54.28%
17/17 [==============================] - 0s 14ms/step
Subject 6 accuracy: 49.72%
16/16 [==============================] - 0s 9ms/step
Subject 7 accuracy: 49.30%
16/16 [==============================] - 0s 8ms/step
Subject 8 accuracy: 53.88%
14/14 [==============================] - 0s 15ms/step
Subject 9 accuracy: 47.24%
Model accuracy percent: 51.99% (var:1.10%)


And that is it. While your results may always differ a bit, we can always reach an accuracy level **better than that described on the EEGNet paper** with EEGNet combined with our method of validation and choice of loss function!

## Bonus: ShallowConvNet and DeepConvNet
I have also implemented other architectures tested against EEGNet on the [paper](https://arxiv.org/abs/1611.08024) so you can play with and compare their performances:

In [10]:
"""
ShallowConvNet
For classification of oscillatory signals

"""

def ShallowConvNet(X : np.ndarray ,n_classes : int) -> keras.Model:
    
    n_channels = X.shape[1]

    input_shape = X.shape[1:]
    
    inputLayer = Input(input_shape)
    
    reshapeLayer = tf.expand_dims(inputLayer,axis=-1)
    
    conv2dLayer_1 = Conv2D(filters = 40, 
                            kernel_size= (1,13), 
                            activation='linear',
                            padding = 'same',
                            kernel_constraint = MaxNorm(max_value = 2))(reshapeLayer) 
    
    conv2dLayer_2 = Conv2D(filters = 40, 
                            kernel_size= (n_channels,1), 
                            activation='linear',
                            padding = 'valid',
                            use_bias = False,
                            kernel_constraint = MaxNorm(max_value = 2))(conv2dLayer_1) 
    
    normLayer = BatchNormalization(epsilon = 1e-5, 
                                   momentum = 0.1)(conv2dLayer_2)
    
    activationLayer_1 = Activation(activation = tf.math.square)(normLayer)
    
    poolLayer = AveragePooling2D(pool_size=(1, 35),strides=(1, 7))(activationLayer_1)
    
    activationLayer_2 = Activation(activation = tf.math.log)(poolLayer)
    
    flatFeatures = Flatten()(activationLayer_2)
    
    dropFeatures = Dropout(rate = 0.5)(flatFeatures)
    
    if n_classes == 2:
        outputLayer = Dense(units = 1,activation='sigmoid',kernel_constraint=MaxNorm(max_value=2))(dropFeatures)
    else:
        outputLayer = Dense(units = n_classes,activation='softmax',kernel_constraint=MaxNorm(max_value=2))(dropFeatures)    
    
    model = keras.Model(inputLayer,outputLayer)
    
    return model

"""
DeepConvNet
For general purpose classification

"""

def DeepConvNet(X : np.ndarray ,n_classes : int) -> keras.Model:
    
    n_channels = X.shape[1]
    
    input_shape = X.shape[1:]

    inputLayer = Input(input_shape)
    
    reshapeLayer = tf.expand_dims(inputLayer,axis=-1)
    
    conv2dLayer_1 = Conv2D(filters = 25, 
                            kernel_size= (1,5), 
                            activation='linear',
                            padding = 'valid',
                            kernel_constraint = MaxNorm(max_value = 2))(reshapeLayer) 
    
    
    ###
    
    conv2dLayer_2 = Conv2D(filters = 25, 
                            kernel_size= (n_channels,1), 
                            activation='linear',
                            padding = 'valid',
                            kernel_constraint = MaxNorm(max_value = 2))(conv2dLayer_1) 
    
    normLayer_1 = BatchNormalization(epsilon = 1e-5, 
                                     momentum = 0.1)(conv2dLayer_2)
    
    activationLayer_1 = Activation(activation = 'elu')(normLayer_1)
    
    poolLayer_1 = MaxPooling2D(pool_size=(1, 2))(activationLayer_1)
    
    dropLayer_1 = Dropout(rate = 0.5)(poolLayer_1)
    
    #####
    
    conv2dLayer_3 = Conv2D(filters = 50, 
                            kernel_size= (1,5), 
                            activation='linear',
                            padding = 'valid',
                            kernel_constraint = MaxNorm(max_value = 2))(dropLayer_1) 
    
    normLayer_2 = BatchNormalization(epsilon = 1e-5, 
                                     momentum = 0.1)(conv2dLayer_3)
    
    activationLayer_2 = Activation(activation = 'elu')(normLayer_2)
    
    poolLayer_2 = MaxPooling2D(pool_size=(1, 2))(activationLayer_2)
    
    dropLayer_2 = Dropout(rate = 0.5)(poolLayer_2)
    
    
    #####
    
    
    conv2dLayer_4 = Conv2D(filters = 100, 
                            kernel_size= (1,5), 
                            activation='linear',
                            padding = 'valid',
                            kernel_constraint = MaxNorm(max_value = 2))(dropLayer_2) 
    
    normLayer_3 = BatchNormalization(epsilon = 1e-5, 
                                     momentum = 0.1)(conv2dLayer_4)
    
    activationLayer_3 = Activation(activation = 'elu')(normLayer_3)
    
    poolLayer_3 = MaxPooling2D(pool_size=(1, 2))(activationLayer_3)
    
    dropLayer_3= Dropout(rate = 0.5)(poolLayer_3)
    
    
    ####
    
    
    conv2dLayer_5 = Conv2D(filters = 200, 
                            kernel_size= (1,5), 
                            activation='linear',
                            padding = 'valid',
                            kernel_constraint = MaxNorm(max_value = 2))(dropLayer_3) 
    
    normLayer_4 = BatchNormalization(epsilon = 1e-5, 
                                     momentum = 0.1)(conv2dLayer_5)
    
    activationLayer_4 = Activation(activation = 'elu')(normLayer_4)
    
    poolLayer_4 = MaxPooling2D(pool_size=(1, 2))(activationLayer_4)
    
    dropLayer_4 = Dropout(rate = 0.5)(poolLayer_4)
    
    
    ####
    
    flatFeatures = Flatten()(dropLayer_4)
    
    if n_classes == 2:
        outputLayer = Dense(units = 1,activation='sigmoid',kernel_constraint=MaxNorm(max_value=2))(flatFeatures)
    else:
        outputLayer = Dense(units = n_classes,activation='softmax',kernel_constraint=MaxNorm(max_value=2))(flatFeatures)

    model = keras.Model(inputLayer,outputLayer)
    
    return model